# TAPT: Test-Time Adversarial Prompt Tuning

This notebook replicates results from the TAPT paper (CVPR 2025).

**Paper**: [TAPT: Test-Time Adversarial Prompt Tuning for Robust Inference in Vision-Language Models](https://arxiv.org/abs/2411.13136)

## Workflow
1. **Setup Environment** - Install dependencies
2. **Mount Google Drive** - Access datasets
3. **Clone Repository** - Get the code
4. **Option A**: Use pre-trained weights (if available)
5. **Option B**: Train AdvIVLP weights on ImageNet (requires ImageNet dataset)
6. **Run TAPT Evaluation** - Evaluate on DTD dataset

## Step 1: Check GPU

In [ ]:
!nvidia-smi
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

## Step 2: Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Step 3: Clone Repository

**TODO**: Replace `YOUR_GITHUB_USERNAME` with your actual GitHub username

In [ ]:
# Clone your repository (replace with your GitHub repo URL)
%cd /content
!git clone https://github.com/YOUR_GITHUB_USERNAME/Baseline.git
%cd Baseline

## Step 4: Setup Environment

In [ ]:
# Install Dassl.pytorch
%cd /content/Baseline/Dassl.pytorch
!pip install -r requirements.txt
!python setup.py develop

In [ ]:
# Install CLIP
!pip install git+https://github.com/openai/CLIP.git

In [ ]:
# Install additional dependencies
!pip install ftfy regex tqdm scipy scikit-learn tabulate yacs gdown future

In [ ]:
# Set environment variables
import os
import sys

# Add paths to Python path
sys.path.insert(0, '/content/Baseline/TAPT')
sys.path.insert(0, '/content/Baseline/Dassl.pytorch')
sys.path.insert(0, '/content/Baseline/Multimodal-Adversarial-Prompt-Tuning')

# Set DATA path
os.environ['DATA'] = '/content/drive/MyDrive/datasets'

print("Environment setup complete!")
print(f"DATA path: {os.environ['DATA']}")

## Step 5: Verify Dataset

Make sure your DTD dataset is structured correctly:
```
/content/drive/MyDrive/datasets/
└── dtd/
    ├── images/
    ├── imdb/
    ├── labels/
    └── split_zhou_DescribableTextures.json
```

In [ ]:
# Verify DTD dataset structure
import os

dtd_path = '/content/drive/MyDrive/datasets/dtd'

if os.path.exists(dtd_path):
    print("DTD dataset found!")
    print("Contents:")
    for item in os.listdir(dtd_path):
        full_path = os.path.join(dtd_path, item)
        if os.path.isdir(full_path):
            print(f"  📁 {item}/")
        else:
            print(f"  📄 {item}")
    
    # Check for split file
    split_file = os.path.join(dtd_path, 'split_zhou_DescribableTextures.json')
    if os.path.exists(split_file):
        print("\n✅ Split file found!")
    else:
        print("\n❌ WARNING: split_zhou_DescribableTextures.json not found!")
        print("Download from: https://drive.google.com/file/d/1u3_QfB467jqHgNXC00UIzbLZRQCg2S7x/view")
else:
    print(f"❌ DTD dataset not found at {dtd_path}")

## Step 6: Verify Imports

In [ ]:
# Test imports
try:
    import torch
    import clip
    from dassl.config import get_cfg_default
    print("✅ All imports successful!")
    print(f"   - PyTorch: {torch.__version__}")
    print(f"   - CLIP models available: {clip.available_models()}")
except ImportError as e:
    print(f"❌ Import error: {e}")

---

# Option A: Using Pre-trained Weights (if available)

If you have pre-trained AdvIVLP weights, upload them to your Google Drive and update the path below.

In [ ]:
# Skip this cell if you don't have pre-trained weights
# This is just to check if weights exist

weights_path = '/content/Baseline/output/train/imagenet/AdvIVLP/vit_b16_c2_ep100_batch32_2+2ctx_9depth_16shots/seed1'

if os.path.exists(weights_path):
    print(f"✅ Pre-trained weights found at: {weights_path}")
    print("Contents:")
    for item in os.listdir(weights_path):
        print(f"  - {item}")
else:
    print("❌ Pre-trained weights not found.")
    print("You need to either:")
    print("  1. Train AdvIVLP weights (Option B below)")
    print("  2. Contact authors for pre-trained weights")

---

# Option B: Train AdvIVLP Weights (Requires ImageNet)

⚠️ **WARNING**: This requires the ImageNet dataset (~150GB) and significant training time.

Training settings:
- 16-shot samples from ImageNet
- 100 epochs
- Batch size: 32 (may need to reduce for T4 GPU)

In [ ]:
# Check if ImageNet is available
imagenet_path = '/content/drive/MyDrive/datasets/imagenet'

if os.path.exists(imagenet_path):
    print(f"✅ ImageNet found at: {imagenet_path}")
    print("You can proceed with training.")
else:
    print("❌ ImageNet not found.")
    print("Training requires ImageNet dataset.")
    print("See DATASETS.md for download instructions.")

In [ ]:
# Train AdvIVLP (only run if ImageNet is available)
# This will take several hours

%cd /content/Baseline/Multimodal-Adversarial-Prompt-Tuning

!python train.py \
    --root /content/drive/MyDrive/datasets \
    --seed 1 \
    --trainer AdvIVLP \
    --dataset-config-file configs/datasets/imagenet.yaml \
    --config-file configs/trainers/AdvIVLP/vit_b16_c2_ep100_batch32_2+2ctx_9depth.yaml \
    --output-dir /content/Baseline/output/train/imagenet/AdvIVLP/vit_b16_c2_ep100_batch32_2+2ctx_9depth_16shots/seed1 \
    DATASET.NUM_SHOTS 16 \
    DATALOADER.TRAIN_X.BATCH_SIZE 16

---

# Step 7: Run TAPT Evaluation on DTD (Clean)

In [ ]:
# Run TAPT-VLI on DTD (Clean evaluation)
# Make sure you have pre-trained weights before running this

%cd /content/Baseline/TAPT

!python train.py \
    --root /content/drive/MyDrive/datasets \
    --seed 1 \
    --trainer TAPTVLI \
    --dataset-config-file configs/datasets/dtd.yaml \
    --config-file configs/trainers/TAPTVLI/TAPT_vit_b16_c2_ep100_batch32_2ctx_9depth_l1_cross_datasets_step1_clean.yaml \
    --output-dir output/TAPTVLI/clean/dtd/seed1/100 \
    --model-dir /content/Baseline/output/train/imagenet/AdvIVLP/vit_b16_c2_ep100_batch32_2+2ctx_9depth_16shots/seed1 \
    --load-epoch 100 \
    --tapt \
    DATASET.NUM_SHOTS 0

## Step 8: View Results

In [ ]:
# View evaluation results
import os

results_dir = '/content/Baseline/TAPT/output/TAPTVLI/clean/dtd/seed1/100'

if os.path.exists(results_dir):
    print(f"Results directory: {results_dir}")
    print("\nContents:")
    for item in os.listdir(results_dir):
        print(f"  - {item}")
    
    # Try to read log file
    log_file = os.path.join(results_dir, 'log.txt')
    if os.path.exists(log_file):
        print("\n" + "="*50)
        print("LOG FILE (last 30 lines):")
        print("="*50)
        with open(log_file, 'r') as f:
            lines = f.readlines()
            for line in lines[-30:]:
                print(line.rstrip())
else:
    print(f"Results not found at {results_dir}")
    print("Make sure to run the evaluation first.")

---

## Troubleshooting

### Common Issues:

1. **Out of Memory (OOM)**: Reduce batch size in the config file or command line
   ```
   DATALOADER.TRAIN_X.BATCH_SIZE 2
   DATALOADER.TEST.BATCH_SIZE 50
   ```

2. **Missing split file**: Download `split_zhou_DescribableTextures.json` from:
   https://drive.google.com/file/d/1u3_QfB467jqHgNXC00UIzbLZRQCg2S7x/view

3. **Import errors**: Make sure to run all setup cells in order

4. **Dataset not found**: Check the path to your Google Drive datasets folder